#### Naive Bayes classifier

With the nearest neighbor algorithms, it is difficult to quantify confidence about a classification. With stats, specifically bayesian methods, we can make probabilistic classifications. (refer to proababilistic programming ipython tutorial for bayesian methods and markov chains. 

Nearest neighbor algorithms are lazy learners. After normalizing or standardizing trained data, they only go through the entire dataset when a vector needs to be classified. Conversely, bayesian methods are eager learners that analyze data & continously improve models. Since a model is already made, classifcation is way faster. (notes for basics of conditional probability not needed).

Classification example using Bayes max(P(gymnast, vector), P(basketball, vector), P(cyclist, vector)) where vector is the additional evidence - maximum a posteriori hyposthesis or $ h_{MAP} $

  $ h_{MAP} = arg max_{h \in H} P(h | D) $ where D is additional data/evidence and h is hypothesis
  
  also, $ h_{MAP} = arg max_{h \in H} P(D | h) * P(h) $ using Bayes' theorem and removing the constant denominator $P(D)$
  
A naive bayes classifier computes P(h | D) and uses $h_{map}$:

  $ P(class | F_{1}, F_{2}, ... , F_{n}) = \large \frac {P(F_{1}, F_{2}, ... , F_{n} | class) \cdot P(class)} {P(F_{1} \cap F_{2} ... \cap F_{n})} $
  
  $ P(class | F_{1}, F_{2}, ... , F_{n}) = P(F_{1}, F_{2}, ... , F_{n} | class) \cdot P(class) $
  
  $ P(class | F_{1}, F_{2}, ... , F_{n}) = P(class \cap F_{1} \cap F_{2} \cap ... F_{n})  $      ---- > (A)
  

$ P(class \cap F_{1} \cap F_{2} \cap ... F_{n}) = P(class) \cdot P(F_{1} | class)  \cdot P(F_{2} | class, F_{1}) \cdot P(F_{3} | class, F_{1}, F_{2}) \cdot ... \cdot P(F_{n} | class, F_{1}, F_{2}, ... , F_{n-1})  $ ------> (B)

if features are independent:

$ P(F_{1}, F_{2}, .. , F_{n} | class) = P(F_{1} | class) \cdot P(F_{2} | class) \cdot .. \cdot P(F_{n} | class) $

Therefore

$ P(class | F_{1}, F_{2}, ... , F_{n}) = P(class) \cdot P(F_{1} | class) \cdot P(F_{2} | class) \cdot .. \cdot P(F_{n} | class) $

$ P(C | F) = P(C) \cdot \prod_{n \in N} {P(F_{x} | C)} $

and classifier function:

$ h_{NB} = argmax_{c \in C} P(C) \cdot \prod_{n \in N} {P(F_{x} | C)} $

In [2]:
house_votes_raw = []
with open('data/ch6_naivebayes/house-votes/hv-01') as f:
    house_votes_raw = f.readlines()
house_votes = map(lambda x: x.strip().split('\t'), house_votes_raw)
house_votes_categories = ['class'] + ['f']*16
print house_votes_categories

['class', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']


In [3]:
# training data for naive bayes in python
# 1. occurences and probability of each class 2. occurences and probability of each feature 3. P(feature|class)

from collections import defaultdict

class BayesClassifier(object):
    #assumes that data is broken into n buckets. use buckets in KNN notebook
    def __init__(self,bucketPrefix,dataFormat,n_buckets):
        #dataFormat in array form
        self.bucketPrefix = bucketPrefix
        self.dataFormat = dataFormat
        self.n_buckets = n_buckets
        self.raw_data = self.get_raw_data()
        self.data = self.clean_raw_data()
        self.priori = self.get_priori()
        self.conditional_buckets = self.get_conditional()
        
        # structure of conditional bucket
        # bucket -> class -> feature_no -> feature -> P(feature | class)
        
    def get_raw_data(self):
        raw = []
        for i in xrange(1,self.n_buckets+1):
            filename, bucket_data = "%s-%02i" % (self.bucketPrefix, i), []
            with open(filename) as f: bucket_data = map(lambda x: x.strip().split('\t'),f.readlines())
            raw.append(bucket_data)
        return raw  
    
    def clean_raw_data(self):
        buckets_data = []
        for b,bucket in enumerate(self.raw_data):
            bucket_data = {}
            for arr in bucket:
                C = ""
                for i,data in enumerate(arr):
                    if self.dataFormat[i] == "class":
                        C = data
                        bucket_data.setdefault(data,{})
                        bucket_data[data].setdefault('count',0)
                        bucket_data[data].setdefault('ignore',[])
                        bucket_data[data]['count']+=1
                for i,data in enumerate(arr):
                    if self.dataFormat[i] == "f":
                        bucket_data[C].setdefault(i,{})
                        bucket_data[C][i].setdefault(data,0)
                        bucket_data[C][i][data]+=1 
                    elif self.dataFormat[i] == "ignore":
                        bucket_data[C]['ignore'].append(data)
            buckets_data.append(bucket_data)
        return buckets_data
    
    def get_priori(self):
        prioris = []
        for i,bucket_data in enumerate(self.data):
            priori, total = defaultdict(dict), 0
            for cls, cls_info in bucket_data.items():
                priori[cls] = self.data[i][cls]['count']
                total += priori[cls]
            for cls in priori.keys(): priori[cls] = priori[cls]/float(total)
            prioris.append(priori)
        return prioris
    
    def get_conditional(self):
        conditional_buckets = []
        for bucket in self.data: conditional_buckets.append(self._get_conditional_for_bucket(bucket))
        return conditional_buckets
    
    def _get_conditional_for_bucket(self, b):
        bucket = b.copy()
        for cls in bucket.keys():
            count = bucket[cls]['count']
            for feature_no, features in bucket[cls].items():
                if isinstance(feature_no, int):
                    for feature in features.keys(): 
                        bucket[cls][feature_no][feature] /= float(count)
        return bucket
    
    def classify(self, vector, bucketNumber):
        data = self.conditional_buckets[bucketNumber]
        priori = self.priori[bucketNumber]
        #print data['republican'][1][vector[0]]
        probabilities = []
        for cls,cls_feature_dict in data.items():
            prob = priori[cls]
            for i,feature in enumerate(vector):
                if feature in cls_feature_dict[i+1].keys():
                    prob *= cls_feature_dict[i+1][feature]
            probabilities.append((prob,cls))
        return sorted(probabilities)[-1][-1]
        
                

In [4]:
# d = BayesClassifier('data/ch6_naivebayes/house-votes/hv',house_votes_categories,10)
# sample_vector = ['democrat', 'y', 'n', 'y', 'n', 'n', 'n', 'y', 'y', 'y', 'n', '', 'n', 'n', 'n', 'y', 'y']
def _testClassifier(bucketPrefix, dataFormat, nBuckets, testBucketNumber):
    d = BayesClassifier(bucketPrefix, dataFormat, nBuckets)
    testBucket = d.raw_data[testBucketNumber]
    correct, total = 0,0 
    for i in range(nBuckets):
        if i == testBucketNumber: continue
        for test in testBucket:
            real = test[0]
            predict = d.classify(test[1:],i)
            total += 1
            if real == predict: correct += 1
    return correct/float(total)

def test(bucketPrefix, dataFormat, nBuckets):
    accuracy = []
    for i in range(nBuckets): accuracy.append(_testClassifier(bucketPrefix, dataFormat, nBuckets, i))
    return accuracy

In [5]:
import pandas
accuracy = test('data/ch6_naivebayes/house-votes/hv', house_votes_categories, 10)
pd_accuracy = pandas.Series(accuracy)
print "Summary of classifier accuracy"
print pd_accuracy.describe()

Summary of classifier accuracy
count    10.000000
mean      0.861171
std       0.080966
min       0.748792
25%       0.831019
50%       0.841586
75%       0.897672
max       0.989899
dtype: float64


the classifier has an accuracy of 86%. This can be improved though. Refer to notebook naïve_bayes2. 